In [7]:
import openai 
from openai import OpenAI
import pandas as pd 
import psycopg2
from vars import conn_params,get_strategy_by_name

In [8]:
conn = psycopg2.connect(**conn_params)  

In [9]:
strategy=get_strategy_by_name(conn,"deafualt choped 10_000 100_000")
strategy

{'strategy_id': 9,
 'source_name': 'deafualt choped 10_000 100_000',
 'description': None,
 'prompt': None,
 'date_created': datetime.datetime(2023, 12, 21, 12, 33, 8, 795371)}

In [11]:
client = OpenAI()

In [12]:
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

In [16]:

def get_gpt_snippets_by_strategy(conn, strategy_id):
    """
    Fetch all GPT snippets associated with a specific strategy ID.

    :param conn: psycopg2 connection object to the database
    :param strategy_id: The strategy ID to filter GPT snippets
    :return: List of dictionaries, each representing a GPT snippet
    """
    with conn.cursor() as cursor:
        cursor.execute("""
            SELECT * FROM GPT_Snippets 
            WHERE strategy_id = %s;
        """, (strategy_id,))

        # Fetch column names from the cursor
        column_names = [desc[0] for desc in cursor.description]

        # Fetch rows and convert each to a dictionary
        snippets = [dict(zip(column_names, row)) for row in cursor.fetchall()]
        return snippets

In [17]:
snippets=get_gpt_snippets_by_strategy(conn,strategy['strategy_id'])
len(snippets)

14380

In [21]:
snippets[1].keys()

dict_keys(['snippet_id', 'snippet_text', 'num_tokens', 'file_id', 'strategy_id'])

In [60]:
completion = client.chat.completions.create(
  #model="gpt-4-32k",
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You will be provided with a text excerpt from the records of Israel's Supreme Court. Your task is to formulate an insightful question that can be uniquely answered by this text."},
    {"role": "user", "content": snippets[1]['snippet_text']}
  ]
)

print(completion.choices[0].message)  # Print the content of the response


ChatCompletionMessage(content="Which specific directive given by the Jerusalem District Court on January 16, 1996, was not appealed in a request for leave to appeal by the respondents in Case No. 3254/96, according to Israel's Supreme Court?", role='assistant', function_call=None)


In [64]:
completion.choices[0].message.content

"Which specific directive given by the Jerusalem District Court on January 16, 1996, was not appealed in a request for leave to appeal by the respondents in Case No. 3254/96, according to Israel's Supreme Court?"

In [40]:
client.__dict__.keys()

dict_keys(['api_key', 'organization', '_version', '_base_url', 'max_retries', 'timeout', '_limits', '_proxies', '_transport', '_custom_headers', '_custom_query', '_strict_response_validation', '_idempotency_header', '_client', '_has_custom_http_client', '_default_stream_cls', 'completions', 'chat', 'edits', 'embeddings', 'files', 'images', 'audio', 'moderations', 'models', 'fine_tuning', 'fine_tunes', 'with_raw_response'])

In [59]:
client.models._get.__dict__.keys()

dict_keys([])

In [65]:
completion = client.chat.completions.create(
  #model="gpt-4-32k",
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You will be provided with a text excerpt from the records of Israel's Supreme Court. Your task is to formulate an insightful question that can be uniquely answered by this text."},
    {"role": "user", "content": snippets[17]['snippet_text']}
  ]
)

print(completion.choices[0].message)  # Print the content of the response


ChatCompletionMessage(content="What specific question can be uniquely answered by the provided text from the records of Israel's Supreme Court?", role='assistant', function_call=None)


In [66]:
snippets[17]['snippet_text']

'בבית המשפט העליון\n בשבתו כבית משפט \nגבוה לצדק\n\nבג"צ\n2534/97\n\nבג"צ\n2535/97\n\nבג"צ\n2541/97\n\nבפני:                                               כבוד\nהמשנה לנשיא ש. לוין\n\n                                                      כבוד\nהשופט א. גולדברג\n\n                                                      כבוד\nהשופט ת. אור\n\n                                                      כבוד\nהשופט י. זמיר\n\n                                                      כבוד\nהשופטת ד. דורנר                                                                                                                                                       \n\nהעותרים\n בבג"צ 2534/97\n:                     \n1. ח"כ יונה יהב\n\n                                                      2.\nח"כ יוסף ביילין\n\nהעותרים\nבבג"צ 2535/97:                     1. ח"כ יוסי שריד\n\n                                                      2.\nח"כ  אמנון רובינשטיין\n\n                                                      3.\nח"כ

In [67]:
completion = client.chat.completions.create(
  #model="gpt-4-32k",
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You will be provided with a text excerpt from the records of Israel's Supreme Court. Your task is to formulate an insightful question that can be uniquely answered by this text."},
    {"role": "user", "content": snippets[17]['snippet_text']},
      {"role": "system", "content": "remember you have to formulate a question"},
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='What was the specific recommendation the State Attorney, Edna Arbel, and the Attorney General, Elyakim Rubinstein, reached regarding the evidence for filing an indictment against Prime Minister Benjamin Netanyahu in the context of the appointment of Roni Bar-On as Attorney General, and why did they decide not to file the indictment?', role='assistant', function_call=None)


In [71]:
completion.usage

CompletionUsage(completion_tokens=65, prompt_tokens=54911, total_tokens=54976)

In [72]:
completion = client.chat.completions.create(
  #model="gpt-4-32k",
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You will be provided with a text excerpt from the records of Israel's Supreme Court. Your task is to formulate an insightful question that can be uniquely answered by this text."},
    {"role": "user", "content": snippets[1]['snippet_text']},
      {"role": "system", "content": "remember you have to formulate a question"},
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Can you identify the main point of contention in Supreme Court case number 7329/96 pertaining to the journalistic privilege in the context of libelous publication allegations involving the newspaper "Maariv"?', role='assistant', function_call=None)


In [73]:
completion.usage

CompletionUsage(completion_tokens=40, prompt_tokens=17061, total_tokens=17101)

In [ ]:
completion = client.chat.completions.create(
  #model="gpt-4-32k",
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You will be provided with a document Your task is to formulate an insightful question that can be uniquely answered by this text."},
    {"role": "user", "content": snippets[1]['snippet_text']},
      {"role": "system", "content": "remember you have to formulate a question"},
  ]
)

print(completion.choices[0].message)

In [75]:
help(client.completions.create)

Help on method create in module openai.resources.completions:

create(*, model: "Union[str, Literal['babbage-002', 'davinci-002', 'gpt-3.5-turbo-instruct', 'text-davinci-003', 'text-davinci-002', 'text-davinci-001', 'code-davinci-002', 'text-curie-001', 'text-babbage-001', 'text-ada-001']]", prompt: 'Union[str, List[str], List[int], List[List[int]], None]', best_of: 'Optional[int] | NotGiven' = NOT_GIVEN, echo: 'Optional[bool] | NotGiven' = NOT_GIVEN, frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[int] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, n: 'Optional[int] | NotGiven' = NOT_GIVEN, presence_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, stop: 'Union[Optional[str], List[str], None] | NotGiven' = NOT_GIVEN, stream: 'Optional[Literal[False]] | Literal[True] | NotGiven' = NOT_GIVEN, suffix: 'Optional[str] | NotGiven' = NOT_GIVEN, temperature: 'Optional